In [ ]:
%pip install /lakehouse/default/Files/unified_etl_core-1.0.0-py3-none-any.whl
%pip install /lakehouse/default/Files/unified_etl_connectwise-1.0.0-py3-none-any.whl

In [2]:
# ---- CELL 2: Environment setup ----
import logging
import os
from datetime import datetime, timedelta

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Set credentials from Key Vault (in production) or environment
os.environ["CW_AUTH_USERNAME"] = "thekking+yemGyHDPdJ1hpuqx"
os.environ["CW_AUTH_PASSWORD"] = "yMqpe26Jcu55FbQk"
os.environ["CW_CLIENTID"] = "c7ea92d2-eaf5-4bfb-a09c-58d7f9dd7b81"

StatementMeta(, feb0af9a-44a1-4d6b-a202-0ddf4d764558, 10, Finished, Available, Finished)

In [ ]:
"""
Incremental data refresh for ConnectWise ETL - Fabric Notebook Version

This code refreshes only recent data instead of re-fetching 5 years worth of records.
Designed to run as a single cell in a Microsoft Fabric notebook.

Prerequisites:
- Run pip install commands in a separate cell first
- Ensure Key Vault secrets are configured (CW_AUTH_USERNAME, CW_AUTH_PASSWORD, CW_CLIENTID)
"""

# Standard imports
import logging

from pyspark.sql import DataFrame

# ETL framework imports
from unified_etl_connectwise import ConnectWiseClient
from unified_etl_connectwise.api_utils import build_condition_string

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def refresh_recent_data(days_back: int = 30) -> dict[str, DataFrame]:
    """
    Refresh only recent data from ConnectWise.

    Strategy:
    - TimeEntry/ExpenseEntry: Only recent entries (by dateEntered)
    - Agreement/PostedInvoice: Only recently updated (by lastUpdated)
    - UnpostedInvoice: ALL records (they're work in progress)

    Args:
        days_back: Number of days to look back (default 30)

    Returns:
        Dictionary mapping entity names to DataFrames
    """
    # Initialize client
    client = ConnectWiseClient()

    # Calculate date threshold
    since_date = (datetime.now() - timedelta(days=days_back)).strftime("%Y-%m-%d")
    print(f"Refreshing data since: {since_date}")

    results = {}

    # Use the existing endpoint mapping from client._get_entity_name
    # Note: ConnectWise API endpoints:
    # - /finance/invoices returns UnpostedInvoice entities
    # - /finance/invoices/posted returns PostedInvoice entities
    endpoints = {
        "TimeEntry": "/time/entries",
        "Agreement": "/finance/agreements",
        "UnpostedInvoice": "/finance/invoices",  # Returns UnpostedInvoice entities
        "PostedInvoice": "/finance/invoices/posted",  # Posted invoices
        "ExpenseEntry": "/expense/entries",
    }

    for entity_name, endpoint in endpoints.items():
        print(f"\nRefreshing {entity_name}...")

        # Build appropriate conditions based on entity type
        if entity_name in ["TimeEntry", "ExpenseEntry"]:
            conditions = build_condition_string(date_entered_gte=since_date)
            order_by = "dateEntered desc"
        else:  # Agreement, PostedInvoice, UnpostedInvoice
            conditions = f"(lastUpdated>=[{since_date}])"
            order_by = "lastUpdated desc"

        try:
            df = client.extract(
                endpoint=endpoint, conditions=conditions, order_by=order_by, page_size=1000
            )

            # Add ETL metadata columns per CLAUDE.md philosophy
            # Note: Legacy tables have old column names, but we should use proper naming
            from pyspark.sql import functions as F

            df = df.withColumn("_etl_timestamp", F.current_timestamp())
            df = df.withColumn("_etl_source", F.lit("connectwise"))
            df = df.withColumn("_etl_batch_id", F.lit(datetime.now().strftime("%Y%m%d_%H%M%S")))

            # Also add legacy columns for backward compatibility with existing schema
            df = df.withColumn("etl_timestamp", F.col("_etl_timestamp").cast("string"))
            df = df.withColumn("etl_entity", F.lit(entity_name))
            df = df.withColumn("etlTimestamp", F.col("_etl_timestamp").cast("string"))
            df = df.withColumn("etlEntity", F.lit(entity_name))

            results[entity_name] = df
            print(f"  Found {df.count()} {entity_name} records")
        except Exception as e:
            print(f"  ERROR extracting {entity_name}: {e}")
            results[entity_name] = None

    return results


def refresh_specific_date_range(start_date: str, end_date: str) -> dict[str, int]:
    """
    Refresh data for a specific date range.

    Args:
        start_date: Start date in YYYY-MM-DD format
        end_date: End date in YYYY-MM-DD format

    Returns:
        Dictionary mapping entity names to record counts
    """
    client = ConnectWiseClient()
    print(f"Refreshing data from {start_date} to {end_date}")

    results = {}

    # Time entries in date range
    conditions = f"dateEntered>=[{start_date}] AND dateEntered<=[{end_date}]"
    time_entries = client.extract(
        endpoint="/time/entries", conditions=conditions, order_by="dateEntered desc", page_size=1000
    )
    results["time_entries"] = time_entries.count()

    return results


def check_latest_records() -> None:
    """
    Check the most recent records to see what dates we have.
    """
    client = ConnectWiseClient()

    # Get just the 10 most recent time entries
    recent_time = client.paginate(
        endpoint="/time/entries",
        entity_name="time_entries",
        fields="id,dateEntered,notes",
        order_by="dateEntered desc",
        max_pages=1,
        page_size=10,
    )

    print("\nMost recent time entries:")
    for entry in recent_time[:5]:
        print(f"  {entry['dateEntered']}: {entry.get('notes', '')[:50]}...")

    # Get most recent invoices
    recent_invoices = client.paginate(
        endpoint="/finance/invoices",
        entity_name="invoices",
        fields="id,invoiceNumber,dateCreated,lastUpdated",
        order_by="lastUpdated desc",
        max_pages=1,
        page_size=10,
    )

    print("\nMost recent invoices:")
    for inv in recent_invoices[:5]:
        print(f"  Invoice keys: {list(inv.keys())[:5]}...")  # Debug line to see actual field names
        print(
            f"  {inv.get('invoiceNumber', inv.get('id', 'N/A'))}: Created {inv.get('dateCreated', inv.get('date', 'N/A'))}, Updated {inv.get('lastUpdated', 'N/A')}"
        )


# ==============================================================================
# FABRIC NOTEBOOK EXECUTION
# ==============================================================================

# BEFORE RUNNING THIS CELL, YOU NEED TO KNOW YOUR TABLE PATHS!
#
# BRONZE → SILVER → GOLD TABLE MAPPING:
#
# Bronze Tables (from API):
#   Agreement        → Silver: Agreement        → Gold: fact_agreement_period
#   TimeEntry        → Silver: TimeEntry        → Gold: fact_time_entry
#   ExpenseEntry     → Silver: ExpenseEntry     → Gold: fact_expense_entry
#   ProductItem      → Silver: ProductItem      → (used in fact_invoice_line)
#   PostedInvoice    → Silver: PostedInvoice    → Gold: fact_invoice_line (requires API permission)
#   UnpostedInvoice  → Silver: UnpostedInvoice  → (may also feed fact_invoice_line)
#
# Gold combines multiple Silver tables:
#   fact_time_entry: TimeEntry + Agreement + Member
#   fact_invoice_line: PostedInvoice/UnpostedInvoice + TimeEntry + ProductItem + Agreement
#
# Based on your SHOW TABLES output, your tables are case-sensitive!

# Configuration - UPDATE THESE BASED ON YOUR ENVIRONMENT
LAKEHOUSE_ROOT = "/lakehouse/default/Tables/"  # Update if different
DAYS_TO_REFRESH = 35  # How many days back to refresh

# Option 1: Check what's the latest data we have
print("=== Checking Latest Records ===")
check_latest_records()

# Option 2: Refresh recent data (recommended for incremental updates)
print(f"\n=== Refreshing Last {DAYS_TO_REFRESH} Days ===")
results = refresh_recent_data(DAYS_TO_REFRESH)
print("\nRefresh Summary:")
for entity, df in results.items():
    if df is not None:
        print(f"  {entity}: {df.count()} records")
    else:
        print(f"  {entity}: ERROR")

# Option 3: Refresh specific date range (uncomment to use)
# start_date = "2024-11-01"
# end_date = "2024-12-31"
# results = refresh_specific_date_range(start_date, end_date)

# Option 4: Write refreshed data to Bronze tables
# The client.extract() already returns validated Spark DataFrames with proper schema

# Initialize client to get spark session
client = ConnectWiseClient()
spark = client.spark

# Debug: Check what tables exist
print("\n=== Checking Bronze Tables ===")
try:
    # List all tables that start with bronze_cw_
    all_tables = spark.sql("SHOW TABLES")
    bronze_tables = all_tables.filter("tableName LIKE 'bronze_cw_%'")
    bronze_tables.show(100, truncate=False)
except Exception as e:
    print(f"Error listing bronze tables: {e}")

# Write the refreshed data to Bronze tables
print("\n=== Writing to Bronze Tables ===")
for entity_name, df in results.items():
    if df is not None and df.count() > 0:
        try:
            # Use the correct table naming convention: bronze_cw_<entity> (lowercase)
            bronze_table = f"bronze_cw_{entity_name.lower()}"

            # The DataFrame from client.extract() is already validated
            # Use append mode with mergeSchema to handle evolution
            df.write.mode("append").option("mergeSchema", "true").saveAsTable(bronze_table)

            print(f"  Appended {df.count()} records to {bronze_table}")

        except Exception as e:
            print(f"  ERROR writing {entity_name}: {e}")

            # For schema merge errors, try to understand the conflict
            if "Failed to merge fields" in str(e) or "DELTA_FAILED_TO_MERGE_FIELDS" in str(e):
                print("  Schema conflict detected. Checking schema differences...")
                try:
                    # Get existing table schema
                    existing_df = spark.table(bronze_table)
                    print(f"\n  Existing {entity_name} schema:")
                    existing_df.printSchema()
                    print(f"\n  New {entity_name} schema:")
                    df.printSchema()
                except Exception as schema_e:
                    print(f"  Could not compare schemas: {schema_e}")

            # Try to understand other errors
            elif "doesn't exist" in str(e) or "Table or view not found" in str(e):
                print(f"  Table {bronze_table} doesn't exist. Creating it...")
                try:
                    df.write.mode("overwrite").saveAsTable(bronze_table)
                    print(f"  Created {bronze_table} with {df.count()} records")
                except Exception as e2:
                    print(f"  ERROR creating table: {e2}")

print("\n=== Incremental Refresh Complete ===")
print("Next steps:")
print("1. Verify the data looks correct")
print("2. Merge (don't overwrite!) the fresh data to Bronze tables")
print("3. Run Silver transformations for the refreshed entities")
print("4. Run Gold transformations (fact tables) that depend on updated Silver tables")

# CASCADE UPDATE EXAMPLE:
# After merging to Bronze, you need to update Silver and Gold layers
#
# # 1. Update Silver layer for refreshed entities
# from unified_etl_core.silver import process_silver_layer
# from unified_etl_connectwise.config import ENTITY_CONFIGS
#
# # Process only the entities we refreshed
# refreshed_entities = [name for name, df in results.items() if df is not None]
# for entity in refreshed_entities:
#     if entity in ENTITY_CONFIGS:
#         config = ENTITY_CONFIGS[entity]
#         process_silver_layer(
#             bronze_table=config["bronze_table"],
#             silver_table=config["silver_table"],
#             # ... other config params
#         )
#
# # 2. Update Gold layer (fact tables that depend on Silver)
# from unified_etl_connectwise.transforms import (
#     create_time_entry_fact,
#     create_invoice_line_fact,
#     create_expense_entry_fact,
#     create_agreement_period_fact
# )
#
# # Update fact tables based on what was refreshed
# if "time_entry" in refreshed_entities or "agreement" in refreshed_entities:
#     # fact_time_entry depends on silver_timeentry + silver_agreement
#     create_time_entry_fact(spark, "silver_", "gold_")
#
# if "invoice" in refreshed_entities or "time_entry" in refreshed_entities:
#     # fact_invoice_line depends on multiple silver tables
#     create_invoice_line_fact(spark, "silver_", "gold_")

In [ ]:
# %%
# ==============================================================================
# CASCADE UPDATE CELL - Run this after successful Bronze refresh
# ==============================================================================
"""
This cell updates Silver and Gold layers after Bronze incremental refresh.
Prerequisites:
- Run the incremental refresh cell above first
- Ensure 'refreshed_entities' variable contains the entities to update
"""

print("\n=== Updating Silver Layer ===")
from unified_etl_connectwise import models
from unified_etl_connectwise.config import SILVER_CONFIG
from unified_etl_core.silver import apply_silver_transformations

# Get the model classes - models is a dict with lowercase keys
model_mapping = {
    "Agreement": models.get("agreement"),
    "TimeEntry": models.get("timeentry"),
    "ExpenseEntry": models.get("expenseentry"),
    "UnpostedInvoice": models.get("invoice"),  # Uses Invoice model
    "PostedInvoice": models.get("invoice"),  # Also uses Invoice model
    "ProductItem": models.get("productitem"),
}

# Process only entities that had new data
refreshed_entities = [name for name, df in results.items() if df is not None and df.count() > 0]
print(f"Entities to update in Silver: {refreshed_entities}")

for entity_name in refreshed_entities:
    # Check if entity has Silver config
    if entity_name in SILVER_CONFIG["entities"]:
        entity_config = SILVER_CONFIG["entities"][entity_name]
        bronze_table = f"bronze_cw_{entity_name.lower()}"
        silver_table = entity_config["silver_table"]  # Use config table name

        print(f"\nProcessing {entity_name}: {bronze_table} -> {silver_table}")

        try:
            # Read Bronze data
            bronze_df = spark.table(bronze_table)
            print(f"  Bronze records: {bronze_df.count()}")

            # Get model class
            model_class = model_mapping.get(entity_name)
            if not model_class:
                print(f"  WARNING: No model class found for {entity_name}")
                continue

            # Apply Silver transformations - use entity_config from SILVER_CONFIG
            silver_df = apply_silver_transformations(
                df=bronze_df,
                entity_config=entity_config,  # Pass the full config
                model_class=model_class,
            )

            # Write to Silver table
            silver_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable(
                silver_table
            )
            print(f"  Updated {silver_table} with {silver_df.count()} records")

        except Exception as e:
            print(f"  ERROR processing {entity_name}: {e}")
    else:
        print(f"\n{entity_name} not found in SILVER_CONFIG, skipping Silver update")

# CASCADE UPDATE - GOLD LAYER
print("\n\n=== Updating Gold Layer ===")
from unified_etl_connectwise.transforms import (
    create_agreement_period_fact,
    create_expense_entry_fact,
    create_invoice_line_fact,
    create_time_entry_fact,
)

# Check which Gold fact tables need updating based on refreshed entities
print("\nDetermining which fact tables to update...")

if "TimeEntry" in refreshed_entities or "Agreement" in refreshed_entities:
    print("\nCreating fact_time_entry (depends on TimeEntry + Agreement)...")
    try:
        # Load required DataFrames - using correct table names
        time_entry_silver = spark.table("silver_cw_timeentry")
        agreement_silver = spark.table("silver_cw_agreement")

        # Check if member table exists (optional for cost data)
        member_silver = None
        if spark.catalog.tableExists("silver_cw_member"):
            member_silver = spark.table("silver_cw_member")
            print("  Found member table for cost enrichment")

        fact_df = create_time_entry_fact(
            spark=spark,
            time_entry_df=time_entry_silver,
            agreement_df=agreement_silver,
            member_df=member_silver,
        )

        # Use overwrite mode since this is a new fact table
        fact_df.write.mode("overwrite").saveAsTable("gold_fact_time_entry")
        print(f"  Created fact_time_entry with {fact_df.count()} records")
    except Exception as e:
        print(f"  ERROR creating fact_time_entry: {e}")

if "Agreement" in refreshed_entities:
    print("\nCreating fact_agreement_period...")
    try:
        agreement_silver = spark.table("silver_cw_agreement")
        fact_df = create_agreement_period_fact(
            spark=spark, agreement_df=agreement_silver, config={}
        )
        fact_df.write.mode("overwrite").saveAsTable("gold_fact_agreement_period")
        print(f"  Created fact_agreement_period with {fact_df.count()} records")
    except Exception as e:
        print(f"  ERROR creating fact_agreement_period: {e}")

if (
    "UnpostedInvoice" in refreshed_entities
    or "PostedInvoice" in refreshed_entities
    or "TimeEntry" in refreshed_entities
):
    print(
        "\nCreating fact_invoice_line (depends on Invoice + TimeEntry + ProductItem + Agreement)..."
    )
    try:
        # Load all required tables
        invoice_silver = spark.table("silver_cw_invoice")
        time_entry_silver = (
            spark.table("silver_cw_timeentry")
            if spark.catalog.tableExists("silver_cw_timeentry")
            else None
        )
        product_silver = (
            spark.table("silver_cw_productitem")
            if spark.catalog.tableExists("silver_cw_productitem")
            else None
        )
        agreement_silver = (
            spark.table("silver_cw_agreement")
            if spark.catalog.tableExists("silver_cw_agreement")
            else None
        )

        fact_df = create_invoice_line_fact(
            spark=spark,
            invoice_df=invoice_silver,
            time_entry_df=time_entry_silver,
            product_df=product_silver,
            agreement_df=agreement_silver,
        )
        fact_df.write.mode("overwrite").saveAsTable("gold_fact_invoice_line")
        print(f"  Created fact_invoice_line with {fact_df.count()} records")
    except Exception as e:
        print(f"  ERROR creating fact_invoice_line: {e}")

if "ExpenseEntry" in refreshed_entities:
    print("\nCreating fact_expense_entry...")
    try:
        expense_silver = spark.table("silver_cw_expenseentry")
        agreement_silver = (
            spark.table("silver_cw_agreement")
            if spark.catalog.tableExists("silver_cw_agreement")
            else None
        )

        fact_df = create_expense_entry_fact(
            spark=spark, expense_df=expense_silver, agreement_df=agreement_silver
        )
        fact_df.write.mode("overwrite").saveAsTable("gold_fact_expense_entry")
        print(f"  Created fact_expense_entry with {fact_df.count()} records")
    except Exception as e:
        print(f"  ERROR creating fact_expense_entry: {e}")

print("\n=== Cascade Update Complete ===")
print("Bronze -> Silver -> Gold pipeline updated with fresh data!")

# %%
# ==============================================================================
# DIMENSION GENERATION CELL - Run after Gold layer update
# ==============================================================================
"""
Generate dimensions from Silver tables for PowerBI consumption.
This creates lightweight dimension tables from enum-like columns.
"""

print("\n=== Generating Dimensions ===")
from unified_etl_core.date_utils import generate_date_dimension
from unified_etl_core.dimensions import create_dimension_from_column

# Generate date dimension if it doesn't exist
if not spark.catalog.tableExists("gold_dim_date"):
    print("\nCreating date dimension...")
    try:
        date_dim = generate_date_dimension(
            spark=spark,
            start_date="2020-01-01",
            end_date="2030-12-31",
            fiscal_year_start_month=7,  # July fiscal year
        )
        date_dim.write.mode("overwrite").saveAsTable("gold_dim_date")
        print(f"  Created dim_date with {date_dim.count()} records")
    except Exception as e:
        print(f"  ERROR creating dim_date: {e}")

# Define dimensions to generate from Silver tables
dimension_configs = [
    # From TimeEntry
    {
        "source_table": "silver.silver_cw_timeentry",
        "column": "billableOption",
        "dimension_name": "dim_billable_option",
        "source": "connectwise",
    },
    {
        "source_table": "silver.silver_cw_timeentry",
        "column": "status",
        "dimension_name": "dim_time_status",
        "source": "connectwise",
    },
    {
        "source_table": "silver.silver_cw_timeentry",
        "column": "chargeToType",
        "dimension_name": "dim_charge_type",
        "source": "connectwise",
    },
    # From Agreement
    {
        "source_table": "silver.silver_cw_agreement",
        "column": "agreementStatus",
        "dimension_name": "dim_agreement_status",
        "source": "connectwise",
    },
    {
        "source_table": "silver.silver_cw_agreement",
        "column": "billCycleIdentifier",
        "dimension_name": "dim_bill_cycle",
        "source": "connectwise",
    },
    {
        "source_table": "silver.silver_cw_agreement",
        "column": "periodType",
        "dimension_name": "dim_period_type",
        "source": "connectwise",
    },
    # From Invoice
    {
        "source_table": "silver.silver_cw_invoice",
        "column": "statusName",
        "dimension_name": "dim_invoice_status",
        "source": "connectwise",
    },
    # From ExpenseEntry
    {
        "source_table": "silver.silver_cw_expenseentry",
        "column": "typeName",
        "dimension_name": "dim_expense_type",
        "source": "connectwise",
    },
]

# Generate dimensions
for config in dimension_configs:
    if spark.catalog.tableExists(config["source_table"]):
        print(
            f"\nGenerating {config['dimension_name']} from {config['source_table']}.{config['column']}..."
        )
        try:
            dim_df = create_dimension_from_column(
                spark=spark,
                source_table=config["source_table"],
                column_name=config["column"],
                dimension_name=config["dimension_name"],
                include_counts=True,
            )

            # Write dimension table
            table_name = f"gold_{config['dimension_name']}"
            dim_df.write.mode("overwrite").saveAsTable(table_name)
            print(f"  Created {table_name} with {dim_df.count()} values")
        except Exception as e:
            print(f"  ERROR creating {config['dimension_name']}: {e}")
    else:
        print(
            f"\nSkipping {config['dimension_name']} - source table {config['source_table']} not found"
        )

print("\n=== Dimension Generation Complete ===")

# %%

In [6]:
# ==============================================================================
# COLUMN VALUE ANALYSIS CELL - Identify high-value vs noise columns
# ==============================================================================
"""
Analyze columns to identify which have signal vs noise for PowerBI views.
This helps create focused views that exclude columns with no analytical value.
"""

print("\n=== Column Value Analysis ===")

from pyspark.sql import functions as F  # noqa: E402


def analyze_column_value(df, table_name, sample_size=10000):
    """Analyze columns to provide detailed statistics without arbitrary scoring."""

    # Sample the data if it's large
    total_count = df.count()
    if total_count > sample_size:
        df_sample = df.sample(fraction=sample_size / total_count)
    else:
        df_sample = df

    print(f"\nAnalyzing {table_name} ({total_count} rows, sampled {df_sample.count()})...")

    analysis_results = []

    for col in df.columns:
        # Skip system columns
        if col.startswith("_etl_") or col.startswith("etl"):
            continue

        # Get basic stats
        distinct_count = df_sample.select(col).distinct().count()
        null_count = df_sample.filter(F.col(col).isNull()).count()
        null_percentage = (null_count / df_sample.count()) * 100
        non_null_count = df_sample.count() - null_count

        # Determine column type
        col_type = str(df.schema[col].dataType)

        # Calculate cardinality ratio (distinct values / non-null rows)
        cardinality_ratio = distinct_count / non_null_count if non_null_count > 0 else 0

        # Get sample values for low-cardinality columns
        sample_values = []
        if distinct_count <= 10 and distinct_count > 0:
            values_df = (
                df_sample.select(col).distinct().filter(F.col(col).isNotNull()).limit(10).collect()
            )
            sample_values = [str(row[0]) for row in values_df]

        # For numeric columns, get min/max/mean
        numeric_stats = {}
        if (
            "Int" in col_type
            or "Double" in col_type
            or "Decimal" in col_type
            or "Float" in col_type
        ):
            stats_df = df_sample.select(
                F.min(col).alias("min"),
                F.max(col).alias("max"),
                F.mean(col).alias("mean"),
                F.stddev(col).alias("stddev"),
            ).collect()[0]
            numeric_stats = {
                "min": stats_df["min"],
                "max": stats_df["max"],
                "mean": round(stats_df["mean"], 2) if stats_df["mean"] else None,
                "stddev": round(stats_df["stddev"], 2) if stats_df["stddev"] else None,
            }

        # Column category based on characteristics
        category = "unknown"
        if col.lower().endswith(("id", "key", "_id", "_key")):
            category = "identifier"
        elif any(
            kw in col.lower() for kw in ["amount", "cost", "price", "revenue", "total", "sum"]
        ):
            category = "financial"
        elif any(kw in col.lower() for kw in ["hours", "days", "minutes", "duration"]):
            category = "time_measure"
        elif any(kw in col.lower() for kw in ["date", "time", "created", "updated", "modified"]):
            category = "temporal"
        elif any(kw in col.lower() for kw in ["name", "description", "notes", "comment"]):
            category = "text"
        elif any(kw in col.lower() for kw in ["status", "type", "category", "class"]):
            category = "categorical"
        elif "Boolean" in col_type:
            category = "boolean"
        elif distinct_count == 1:
            category = "constant"
        elif cardinality_ratio > 0.95:
            category = "high_cardinality"
        elif cardinality_ratio < 0.01:
            category = "low_cardinality"

        analysis_results.append(
            {
                "column": col,
                "type": col_type,
                "category": category,
                "distinct_values": distinct_count,
                "null_count": null_count,
                "null_percentage": round(null_percentage, 1),
                "cardinality_ratio": round(cardinality_ratio, 3),
                "sample_values": sample_values[:5] if sample_values else [],
                "numeric_stats": numeric_stats,
            }
        )

    # Sort by category, then by null percentage
    analysis_results.sort(key=lambda x: (x["category"], x["null_percentage"]))

    # Print analysis by category
    categories = {}
    for result in analysis_results:
        cat = result["category"]
        if cat not in categories:
            categories[cat] = []
        categories[cat].append(result)

    print("\n  Column Analysis by Category:")
    print(f"  {'Category':<20} {'Count':<8} {'Avg Nulls %':<12}")
    print(f"  {'-' * 40}")

    for cat, cols in categories.items():
        avg_nulls = sum(c["null_percentage"] for c in cols) / len(cols)
        print(f"  {cat:<20} {len(cols):<8} {avg_nulls:<12.1f}")

    # Show detailed analysis for each category
    for cat, cols in categories.items():
        print(f"\n  === {cat.upper()} COLUMNS ===")
        for col in cols[:5]:  # Show first 5 of each category
            print(f"\n  {col['column']} ({col['type']})")
            print(
                f"    Nulls: {col['null_percentage']}% | Distinct: {col['distinct_values']} | Cardinality: {col['cardinality_ratio']}"
            )
            if col["sample_values"]:
                print(f"    Sample values: {col['sample_values']}")
            if col["numeric_stats"]:
                stats = col["numeric_stats"]
                print(
                    f"    Range: [{stats['min']} - {stats['max']}] | Mean: {stats['mean']} | StdDev: {stats['stddev']}"
                )

        if len(cols) > 5:
            print(f"    ... and {len(cols) - 5} more {cat} columns")

    # Identify potentially problematic columns
    print("\n  === POTENTIAL ISSUES ===")

    constant_cols = [r for r in analysis_results if r["category"] == "constant"]
    if constant_cols:
        print("\n  Constant columns (single value):")
        for col in constant_cols:
            print(
                f"    {col['column']}: '{col['sample_values'][0] if col['sample_values'] else 'NULL'}'"
            )

    high_null_cols = [r for r in analysis_results if r["null_percentage"] > 95]
    if high_null_cols:
        print("\n  Nearly empty columns (>95% null):")
        for col in high_null_cols:
            print(f"    {col['column']}: {col['null_percentage']}% null")

    return analysis_results


# Analyze key tables
tables_to_analyze = [
    "silver_cw_timeentry",
    "silver_cw_agreement",
    "silver_cw_invoice",
    "gold_fact_time_entry",
]

all_analysis = {}
for table in tables_to_analyze:
    if spark.catalog.tableExists(table):
        df = spark.table(table)
        analysis = analyze_column_value(df, table)
        all_analysis[table] = analysis

# %%

StatementMeta(, feb0af9a-44a1-4d6b-a202-0ddf4d764558, 14, Finished, Available, Finished)


=== Column Value Analysis ===


In [ ]:
# ==============================================================================
# CREATE POWERBI VIEWS CELL - Create optimized views for PowerBI
# ==============================================================================
"""
Create SQL views optimized for PowerBI consumption.
These views exclude low-value columns identified in the analysis above.
"""

print("\n=== Creating PowerBI Optimized Views ===")


# Function to create view based on column analysis
def create_powerbi_view(table_name, analysis_results, view_suffix="_pbi"):
    """Create a view excluding problematic columns based on analysis."""

    # Exclude columns based on objective criteria
    exclude_categories = ["constant", "unknown"]  # Always exclude these
    exclude_columns = set()

    for result in analysis_results:
        # Exclude constant columns (single value)
        if result["category"] == "constant" or (result["null_percentage"] > 95 and result["category"] not in [
            "identifier",
            "financial",
        ]) or (result["category"] == "unknown" and result["null_percentage"] > 80):
            exclude_columns.add(result["column"])

    # Get columns to keep
    keep_columns = [r["column"] for r in analysis_results if r["column"] not in exclude_columns]

    # Always include certain critical columns even if they have issues
    critical_patterns = ["id", "key", "sk", "date", "amount", "revenue"]
    for result in analysis_results:
        col = result["column"]
        if any(pattern in col.lower() for pattern in critical_patterns) and col not in keep_columns:
            keep_columns.append(col)

    # Build SELECT statement
    select_cols = ", ".join(keep_columns)
    view_name = f"{table_name}{view_suffix}"

    create_view_sql = f"""
    CREATE OR REPLACE VIEW {view_name} AS
    SELECT {select_cols}
    FROM {table_name}
    """

    try:
        spark.sql(create_view_sql)
        print(
            f"  Created view {view_name} with {len(keep_columns)} columns (excluded {len(exclude_columns)} columns)"
        )
        if exclude_columns:
            print(f"    Excluded: {', '.join(sorted(exclude_columns))}")
        return view_name
    except Exception as e:
        print(f"  ERROR creating view {view_name}: {e}")
        return None


# Create views for analyzed tables
for table_name, analysis in all_analysis.items():
    create_powerbi_view(table_name, analysis)

# Create specialized fact views with business logic
print("\n\nCreating specialized business views...")

# Time Entry Analysis View
time_entry_view_sql = """
CREATE OR REPLACE VIEW v_time_entry_analysis AS
SELECT
    TimeEntrySK,
    WorkDateSK,
    memberName,
    agreementName,
    agreementType,
    utilizationType,
    actualHours,
    potentialRevenue,
    actualCost,
    margin,
    marginPercentage,
    isInternalWork,
    isTimapottur,
    effectiveBillingStatus,
    daysSinceWork,
    daysUninvoiced
FROM gold_fact_time_entry
WHERE actualHours > 0  -- Filter out zero-hour entries
"""

try:
    spark.sql(time_entry_view_sql)
    print("  Created v_time_entry_analysis")
except Exception as e:
    print(f"  ERROR creating v_time_entry_analysis: {e}")

print("\n=== PowerBI View Creation Complete ===")
print("\nNext steps:")
print("1. Connect PowerBI to the Lakehouse")
print("2. Use the _pbi views for optimized performance")
print("3. Use v_time_entry_analysis for time tracking analytics")
print("4. Join views with gold_dim_* tables for descriptive labels")

StatementMeta(, feb0af9a-44a1-4d6b-a202-0ddf4d764558, -1, Cancelled, , Cancelled)